In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import torchvision.utils as utils
import os

In [2]:
#dataset path
data_path_train = "./data/training"
data_path_test = "./data/testing"

In [3]:
# data transform, you can add different transform methods and resize image to any size
img_size = 224
transform = transforms.Compose([
                       transforms.Resize((img_size,img_size)),
                       transforms.ToTensor()
                       ])

#build dataset
dataset = datasets.ImageFolder(root=data_path_train,transform=transform)

# spilt your data into train and val
TOTAL_SIZE = len(dataset)
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])

#build dataloader
train_data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=True,  num_workers=4)

#check dataset
print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 1646
    Root location: ./data/training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
{'Baked Potato': 0, 'Crispy Chicken': 1, 'Donut': 2, 'Fries': 3}


/home/NFS/opt/anaconda3/envs/ml2023f/lib/python3.8/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
#train function
def train(model, criterion, optimizer):
    model.train()
    total_loss = 0.0
    total_correct = 0
    
    # Iterate over data
    for inputs, labels in train_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize
        loss.backward()
        optimizer.step()

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)
        
    avg_loss = total_loss / len(train_data_loader)
    accuracy = total_correct.double() / len(train_dataset) * 100

    print('Training Accuracy: {:.4f}% Training Loss: {:.4f}'.format(accuracy, avg_loss))
    return 

#validation function
def valid(model, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    
    # Iterate over data
    for inputs, labels in val_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)
        
    avg_loss = total_loss / len(val_data_loader)
    accuracy = total_correct.double() / len(val_dataset) * 100

    print('Validation Accuracy: {:.4f}% Validation Loss: {:.4f}'.format(accuracy, avg_loss))
    return accuracy

In [5]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if(torch.cuda.is_available()):
    print("using gpu")
#build your model here

class RN18(nn.Module):
    def __init__(self):
        super(RN18,self).__init__()
        self.conv1 = nn.Conv2d(3, 64 , kernel_size = 7, stride = 2, padding = 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding =1)
        self.Conv1 = nn.Sequential(
            nn.Conv2d(64,64,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64,64,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
        )
        self.Conv2 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size = 3, stride = 2,padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128,128,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
        )
        self.shortcut1 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size = 1, stride = 2),
            nn.BatchNorm2d(128),
        )
        self.Conv3 = nn.Sequential(
            nn.Conv2d(128,128,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128,128,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
        )
        self.Conv4 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size = 3, stride = 2, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256,256,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
        )
        self.shortcut2 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size = 1, stride = 2),
            nn.BatchNorm2d(256),
        )
        self.Conv5 = nn.Sequential(
            nn.Conv2d(256,256,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256,256,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
        )
        self.Conv6 = nn.Sequential(
            nn.Conv2d(256,512,kernel_size = 3, stride = 2, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512,512,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
        )
        self.shortcut3 = nn.Sequential(
            nn.Conv2d(256,512,kernel_size = 1, stride = 2),
            nn.BatchNorm2d(512),
        )
        self.Conv7 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512,512,kernel_size = 3, stride = 1,padding = 1),
            nn.BatchNorm2d(512),
        )
        self.avgpool = nn.AvgPool2d(kernel_size = 7, stride = 1)
        self.FC = nn.Linear(512,1000)
        self.Fc2 = nn.Linear(1000,4)
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.maxpool(x)
        x = self.relu1(self.Conv1(x) + x)
        x = self.relu1(self.Conv1(x) + x)
        x = self.relu1(self.Conv2(x) + self.shortcut1(x))
        x = self.relu1(self.Conv3(x) + x)
        x = self.relu1(self.Conv4(x) + self.shortcut2(x))
        x = self.relu1(self.Conv5(x) + x)
        x = self.relu1(self.Conv6(x) + self.shortcut3(x))
        x = self.relu1(self.Conv7(x) + x)
        x = self.avgpool(x)
        x = x.reshape(x.size(0),-1)
        x = self.FC(x)
        x = self.Fc2(x)
        return x
        
        

#call model
model = RN18()


using gpu


In [6]:
####################  implement your optimizer ###################################
## you can use any training methods if you want (ex:lr decay, weight decay.....)
learning_rate = 3e-4
epochs = 100
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()


# start training
model.to(device=device)
acc_best = 0.0

print('--------------start training--------------')
for epoch in range(1, epochs+1):
    
    print('epoch:', epoch)
    train(model, criterion, optimizer)
    accuracy = valid(model, criterion)
    
    if accuracy > acc_best:
        acc_best = accuracy
        print("model saved")
        # save the model
        torch.save(model, "model.pth")

--------------start training--------------
epoch: 1
Training Accuracy: 26.8062% Training Loss: 1.3853
Validation Accuracy: 32.1212% Validation Loss: 1.3726
model saved
epoch: 2
Training Accuracy: 36.1918% Training Loss: 1.3542
Validation Accuracy: 35.7576% Validation Loss: 1.3522
model saved
epoch: 3
Training Accuracy: 41.7286% Training Loss: 1.3376
Validation Accuracy: 39.3939% Validation Loss: 1.3308
model saved
epoch: 4
Training Accuracy: 44.7670% Training Loss: 1.3184
Validation Accuracy: 38.7879% Validation Loss: 1.3275
epoch: 5
Training Accuracy: 45.0371% Training Loss: 1.3000
Validation Accuracy: 44.2424% Validation Loss: 1.3110
model saved
epoch: 6
Training Accuracy: 48.8859% Training Loss: 1.2784
Validation Accuracy: 46.0606% Validation Loss: 1.3011
model saved
epoch: 7
Training Accuracy: 49.9662% Training Loss: 1.2539
Validation Accuracy: 49.0909% Validation Loss: 1.2243
model saved
epoch: 8
Training Accuracy: 49.0209% Training Loss: 1.2297
Validation Accuracy: 50.3030% Valid